In [1]:
import numpy as np
import pandas as pd
import torch
import transformers

In [2]:
from sklearn.metrics import accuracy_score, f1_score
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
pd.set_option('display.max_colwidth', 4000)
DEBUG = False # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [3]:
tok = T5Tokenizer.from_pretrained('t5-base') # large = out of memory on optim.step()
mdl = T5ForConditionalGeneration.from_pretrained('t5-base').to('cuda')

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
train_df = pd.read_csv('kaggle_tweets_data/train.csv')[['id', 'text', 'target']]
test_df = pd.read_csv('kaggle_tweets_data/test.csv')[['id', 'text']]

In [5]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=1, stratify=train_df['target'])

#train_df, valid_df = train_test_split(train_df, test_size=0.01, random_state=1, stratify=train_df['target'])

In [6]:
prefix = 'is_disaster: '
suffix = ' </s>'

train_df['text'] = train_df['text'].apply(lambda x: prefix + x + suffix)
if DEBUG:
    train_df = train_df.sample(frac=0.25)
valid_df['text'] = valid_df['text'].apply(lambda x: prefix + x + suffix)
if DEBUG:
    valid_df = valid_df.sample(frac=0.25)
test_df['text'] = test_df['text'].apply(lambda x: prefix + x + suffix)

In [7]:
train_ls = train_df['text'].tolist()
train_vals = train_df['target'].apply(lambda x: str(x) + suffix).tolist()
valid_ls = valid_df['text'].tolist()
valid_vals = valid_df['target'].apply(lambda x: str(x) + suffix).tolist()
test_ls = test_df['text'].tolist()
#test_vals = test_df['target'].apply(lambda x: str(x) + suffix).tolist()

In [8]:
class TweetsDataset(Dataset):
    
    def __init__(self, input_ids, attention_mask, decoder_ids, decoder_attention_mask):
        assert input_ids.shape == attention_mask.shape
        assert decoder_ids.shape == decoder_attention_mask.shape
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.decoder_ids = decoder_ids
        self.decoder_attention_mask = decoder_attention_mask
    
    def __getitem__(self, index):
        return (self.input_ids[index], self.attention_mask[index], self.decoder_ids[index], self.decoder_attention_mask[index])
    
    def __len__(self):
        return len(self.input_ids)

In [9]:
train_dict = tok(train_ls, max_length=512, truncation=True, padding=True, return_tensors='pt')
train_label_dict = tok(train_vals, return_tensors='pt', padding=True, truncation=True)

train_input_ids = train_dict['input_ids']
train_attention_mask = train_dict['attention_mask']
train_decoder_ids = train_label_dict['input_ids']
train_decoder_attention_mask = train_label_dict['attention_mask']

train_dataset = TweetsDataset(train_input_ids, train_attention_mask, train_decoder_ids, train_decoder_attention_mask)
train_dl = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [10]:
# train_df['message'].apply(lambda x: len(x)).describe()

In [11]:
valid_dict = tok(valid_ls, max_length=512, truncation=True, padding=True, return_tensors='pt')
valid_label_dict = tok(valid_vals, return_tensors='pt', padding=True, truncation=True)

valid_input_ids = valid_dict['input_ids']
valid_attention_mask = valid_dict['attention_mask']
valid_decoder_ids = valid_label_dict['input_ids']
valid_decoder_attention_mask = valid_label_dict['attention_mask']

valid_dataset = TweetsDataset(valid_input_ids, valid_attention_mask, valid_decoder_ids, valid_decoder_attention_mask)
valid_dl = DataLoader(valid_dataset, batch_size=4, shuffle=True)

In [12]:
#%%time
lr = 1e-3
mdl = T5ForConditionalGeneration.from_pretrained('t5-base').to('cuda')

optim = torch.optim.Adam(mdl.parameters(), lr=lr)
epochs = 10

warmup_epochs = 2

for epoch in range(epochs):
    mdl.train()
    lss = []
    for input_ids, attention_mask, decoder_ids, decoder_attention_mask in train_dl:
        # print(input_ids.shape, attention_mask.shape)
        optim.zero_grad()
        # delete attention mask? attention_mask=attention_mask.to('cuda'), 
        res = mdl(input_ids.to('cuda'), labels=decoder_ids.to('cuda')) #decoder_input_ids=decoder_ids.to('cuda'), decoder_attention_mask=decoder_attention_mask.to('cuda'))
        loss = res[0]
        optim.zero_grad()
        loss.backward()
        optim.step()
        lss.append(int(res[0].cpu().detach().numpy()))
        # break;
    print(f'Training loss: {np.array(lss).mean()}')    
    
    mdl.eval()
    y = []
    y_pred = []
    
    if (epoch < warmup_epochs): # N
        print(f'Epoch: {epoch+1}; validation skipped')
        continue;
        
    with torch.no_grad():
        for input_ids, attention_mask, correct_labels, _ in train_dl:
            res = mdl.generate(input_ids=input_ids.to('cuda'), attention_mask=attention_mask.to('cuda'))
            for elem, lbl in zip(res, correct_labels):
                y_pred.append(int(tok.decode(elem)))
                y.append(int(tok.decode(lbl)))
            # print(res.shape)
    print(f'End of epoch {epoch+1}')
    print(f'Train accuracy: {accuracy_score(y, y_pred)}')
    #print(f'F1 score: {f1_score(y, y_pred, average="micro")}')
    
    y = []
    y_pred = []

    with torch.no_grad():
        for input_ids, attention_mask, correct_labels, _ in valid_dl:
            res = mdl.generate(input_ids=input_ids.to('cuda'), attention_mask=attention_mask.to('cuda'))
            for elem, lbl in zip(res, correct_labels):
                y_pred.append(int(tok.decode(elem)))
                y.append(int(tok.decode(lbl)))
            # print(res.shape)
    #print(f'End of epoch {epoch+1}')
    print(f'Validation accuracy: {accuracy_score(y, y_pred)}')
    #print(f'F1 score: {f1_score(y, y_pred, average="micro")}')
    if (epoch > warmup_epochs): # N
        lr = lr/2
        for g in optim.param_groups:
            g['lr'] = lr

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training loss: 0.013131976362442548
Epoch: 1; validation skipped
Training loss: 0.017071569271175313
Epoch: 2; validation skipped
Training loss: 0.0
End of epoch 3
Train accuracy: 0.8167487684729065
Validation accuracy: 0.7839789888378201
Training loss: 0.0
End of epoch 4
Train accuracy: 0.8206896551724138
Validation accuracy: 0.7866053841103086
Training loss: 0.0
End of epoch 5
Train accuracy: 0.8400656814449918
Validation accuracy: 0.7872619829284307
Training loss: 0.0
End of epoch 6
Train accuracy: 0.8599343185550082
Validation accuracy: 0.8056467498358503
Training loss: 0.0
End of epoch 7
Train accuracy: 0.8642036124794745
Validation accuracy: 0.8036769533814839
Training loss: 0.0
End of epoch 8
Train accuracy: 0.8766830870279146
Validation accuracy: 0.804333552199606
Training loss: 0.0
End of epoch 9
Train accuracy: 0.8768472906403941
Validation accuracy: 0.804333552199606
Training loss: 0.0
End of epoch 10
Train accuracy: 0.877504105090312
Validation accuracy: 0.8082731451083388


In [13]:
torch.save(mdl.state_dict(), 'models/mdl_80train_10epochs_lr1e3_05decay.pkl')

In [14]:
#tok.decode(res[0])

In [15]:
#mdl.forward?
#torch.save(mdl.state_dict(), 'models/mdl_first_try.pkl--')

In [16]:
#y = np.array(y)
#y_pred = np.array(y_pred)

In [17]:
#pd.DataFrame(y==y_pred).value_counts()

In [14]:
class TestTweetsDataset(Dataset):
    
    def __init__(self, input_ids, attention_mask):
        assert input_ids.shape == attention_mask.shape
        self.input_ids = input_ids
        self.attention_mask = attention_mask
    
    def __getitem__(self, index):
        return (self.input_ids[index], self.attention_mask[index])
    
    def __len__(self):
        return len(self.input_ids)

In [15]:
test_dict = tok(test_ls, max_length=512, truncation=True, padding=True, return_tensors='pt')

test_input_ids = test_dict['input_ids']
test_attention_mask = test_dict['attention_mask']

test_dataset = TestTweetsDataset(test_input_ids, test_attention_mask)
test_dl = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [16]:
# testing model

#
#mdl.load_state_dict(torch.load('models/mdl_80train_10epochs_lr1e3_no_decay.pkl'))
mdl.eval()
test_res = []

with torch.no_grad():
    for input_ids, attention_mask in test_dl:
        res = mdl.generate(input_ids=input_ids.to('cuda'), attention_mask=attention_mask.to('cuda'))
        for elem in input_ids:
            print(tok.decode(elem))
        for elem in res:
            test_res.append(tok.decode(elem))
            print(tok.decode(elem))

is_disaster: Just happened a terrible car crash
is_disaster: Heard about #earthquake is different cities, stay safe everyone.
is_disaster: there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all
is_disaster: Apocalypse lighting. #Spokane #wildfires
1
0
1
1
is_disaster: Typhoon Soudelor kills 28 in China and Taiwan
is_disaster: We're shaking...It's an earthquake
is_disaster: They'd probably still show more life than Arsenal did yesterday, eh? EH?
is_disaster: Hey! How are you?
1
1
0
0
is_disaster: What a nice hat?
is_disaster: Fuck off!
is_disaster: No I don't like cold!
is_disaster: NOOOOOOOOO! Don't do that!
0
0
0
0
is_disaster: No don't tell me that!
is_disaster: What if?!
is_disaster: Awesome!
is_disaster: Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU
0
0
0
0
is_disaster: @sunkxssedharry will you wear shorts for race ablaze?
is_disaster: #PreviouslyOnDoyinTv: Toke Maki

is_disaster: @margaretcho Call me a fag and I'm going to call you an ambulance :) #RainbowPower
is_disaster: New Nanotech Device Will Be Able To Target And Destroy Blood Clots http://t.co/MnmyJXQ9go #science
is_disaster: @vballplaya2296 want me to send you some medic supplies from my dad's ambulance?
is_disaster: 2 held with heroin in ambulance http://t.co/d9sOwi1G21 http://t.co/OPxmdPIwAu
0
0
0
1
is_disaster: http://t.co/B2FaSrt1tN Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/O8BJXnEPQm
is_disaster: Why should a helicopter ambulance ride to transfer to a hospital 21 miles away cost $29800? http://t.co/7ZqIreY7Te http://t.co/9Qp3PqPwZv
is_disaster: so privileged and proud to wear this uniform.?? #NHS #Ambulance #GayUK #Uniform #Proud #Privileged #WhatsYourEmergency http://t.co/0BkmuhYSFx
is_disaster: The bartender at work described a drunk man as annihilated @kdunning1919 @hsnowberger @gabrielasmith29. 16 more days
1
1
0
0
is_disaster: Cop pulls drunk dr

is_disaster: One Direction Is my pick for http://t.co/q2eBlOKeVE Fan Army #Directioners http://t.co/eNCmhz6y34 x1435
is_disaster: One Direction Is my pick for http://t.co/q2eBlOKeVE Fan Army #Directioners http://t.co/eNCmhz6y34 x1415
is_disaster: Check out more data on Upper Wabash reservoirs here on @LouisvilleUSACE site: http://t.co/hqqLQUqZmD
is_disaster: ANSWER: 'Therefore it came to pass that in my sixteenth year I did go forth at the head of an army of the... http://t.co/uuAAsb394n
0
0
1
0
is_disaster: One Direction Is my pick for http://t.co/q2eBlOKeVE Fan Army #Directioners http://t.co/eNCmhz6y34 x1424
is_disaster: 5 Seconds of Summer Is my pick for http://t.co/qcHV3JqOVK Fan Army #5SOSFAM http://t.co/gc0uDfnFgg  ⁇ 2
is_disaster: Cheesehead Report - Arson charges filed in Jackson County house fire http://t.co/I3Y1ZWjBzO
is_disaster: Arson suspect linked to 30 fires caught in Northern California http://t.co/z8tlcAdKOw
0
0
1
1
is_disaster: Israeli police unable to solve the case 

is_disaster: Musician Kalle Mattson Recreates 34 Classic Album Covers in Clever Music Video for 'Avalanche' http://t.co/97eWeUWybf
is_disaster: 2 TIX 10/3 Frozen Fury XVII: Los Angeles Kings v Avalanche 216 Row:L MGM Grand http://t.co/qyCwho8guN
is_disaster: FC Maid Agency presents Musician Kalle Mattson Recreates 34 Classic Album Covers in Clever Music Video for... http://t.co/BObrp2qwCb
is_disaster: 02 03 04 05 AVALANCHE 1500 REAR AXLE ASSEMBLY 2055271 http://t.co/VxZhZsAlra http://t.co/HmXWRkbLS0
0
0
0
1
is_disaster: Senators have another year to determine Hoffman's v... #ColoradoAvs #NHLAvalanche http://t.co/CIzZOc6f0D http://t.co/PEmYHD3Nfz
is_disaster: @Mano_twits @Kolaveriboy avast an avalanche of andals
is_disaster: Colorado Avalanche @ Anaheim Ducks 1-7 tickets 10/1 preseason http://t.co/XRU1WowZYG
is_disaster: @90sAngelicMalik listen to wtm avalanche and take a deep breath
0
0
0
0
is_disaster: Now there's an avalanche... These men are lunatics #livingontheedge
is_disaster: #e

is_disaster: did leona lewis ever see that video of dinah singing bleeding love?
is_disaster: @TomBrevoort 'Bleeding Cool as read by Tom Brevoort. Though he may just look at the pictures.'
is_disaster: You cut me open and I keep bleeding. #BleedingLove #LeonaLewis #acapella #singer #canadiansinger ⁇ _ https://t.co/51pfEIlPNK
is_disaster: *Ears bleeding from the bass* https://t.co/d5RrrwHjpN
0
0
0
0
is_disaster: @KrisZellner But Triplemania has the promise of Villano bleeding and Del Rio whooping that Trick Myztiziziziziz.
is_disaster: Looks like Reynolds and Montano coming in. Need to stop the bleeding. Not the best performance so far.
is_disaster: @imKanoMate [the stabbing did not really affect her but she did get stab wounds and was bleeding Skarlet then pushed him backwards +
is_disaster: Yeah I hate cats I just tried to pet our own cat and now my hand has bite marks scratch marks and it's bleeding. #DogsAreBetterThanCats
0
0
0
0
is_disaster: @Aichmomanic wraps a cloth on my wound b

...

is_disaster: @dead_last I'm going to live out of my car for a year probably 2016 life goals :)
is_disaster: #TornadoGiveaway  ⁇  #thebookclub  ⁇  Join in!! https://t.co/8qucf6sgOc https://t.co/5U1t8hc4uf
is_disaster: Loving you was like throwing a lasso around a tornado I tried to hold on to.??
is_disaster: Blonde beauty Amanda shows you her big natural boobs in her first set http://t.co/qew4c5M1xd View and download video
0
0
0
0
is_disaster: Rly tragedy in MP: Some live to recount horror:  ⁇ When I saw coaches of my train plunging into water I called... http://t.co/KiXRcBNjLs
is_disaster: @enjoltair that would be both a tragedy but also extremely realistic and p cool yeah sure
is_disaster: @TEDMED I remember my friend Jeff Weisberg saying  ⁇ If I can only gat back from this ER shift w/out a kiddie tragedy'
is_disaster: '1 Death is a tragedy; a million Deaths is a statistic.' Stalin August 6 1945 U.S. Atomic Bomb Dropped On... http://t.co/kaMPx4eOS9
1
0
0
1
is_disaster: They can only b

is_disaster: Still traumatised at the sight of an old woman throwing her shit out the window?????
is_disaster: for an extra month I was literally traumatised that couldn't been me. But I do a recap in a pool every now &amp; again for safety purposes.
is_disaster: sunakawa is gonna be like traumatised forever i think lol
is_disaster: @lovestheworld55 @SemiMooch so that's why you're now willing to believe nonsense - you were traumatised? Sorry to hear but bible still fake
0
0
0
0
is_disaster: @DanDoherty_ @TomCenci @HRWright too traumatised to ask why.....
is_disaster: @ibAUTUMN I'm that traumatised that I can't even spell properly! Excuse the typos!
is_disaster: @TitusOReily He's not responsible for the trouble urine!
is_disaster: Pissed something off in the wood pile. Pulling pieces out and I get a small hiss and a series of low clicks for my trouble.
0
0
0
0
is_disaster: RT MMDA: ADVISORY: Stalled Bus at EDSA Service Road Cubao SB due to mechanical trouble as of 7:53 AM. 1 lane occupi

is_disaster: Storm is here! Violent winds and pounding rains in Evergreen. #yyc
is_disaster: Stay inside for the next little while kids. We're having a bit of a violent storm right now. -kjc
is_disaster: Watch: Violent Storm Causes Deadly Accident at New Hampshire Circus http://t.co/jpPiR3ZzKA #GMA http://t.co/nV5GCDpIBA
is_disaster: Violent Forces Radio: Now Playing Axegressor - Psalm Before the Storm TuneIn Player @ http://t.co/XsSgEdSbH4
1
1
1
0
is_disaster: #calgaryweather It would be nice if they would fix radar before another violent storm Uninformed citizens dangerous #YYC #yycweather
is_disaster: 2012#Shell's 250-foot-tall drilling rig broke loose from a towline&amp;drifted out of control during violent winter storm #incompetent #shellno
is_disaster: http://t.co/16EClWrW84 Asics GT-II Super Red 2.0 11 Ronnie Fieg Kith Red White 3M x gel grey volcano 2
is_disaster: Force Factor VOLCANO 120 Capsule Brand New - Full read by eBay http://t.co/HewutDLDqh http://t.co/8etXXzBdua
1
1
0


1
1
1
1
is_disaster: 11:57am Wildfire by The Mynabirds from Lovers Know
is_disaster: Finger Rock wildfire grows to 500 acres #Tucson http://t.co/zSXGEI1Nxf
is_disaster: #chattanooga The Latest: California governor headed to site of wildfire http://t.co/D6E7IICW5W
is_disaster: About to get it with the babes of #wildfire and @taviiikenney of course!!
1
1
1
1
is_disaster: Here's a quick timelapse I made of the Finger Rock Fire last night from about 9PM - 1AM. Check it out! #fingerrockfire #wildfire #catalinas
is_disaster: #WAwildfire in #chelan? Wolverine fire results in evacuation. http://t.co/yePlnZPoWu
is_disaster: can't eat a wildfire chicken salad without thinking of / missing @Alynacarol??
is_disaster: Cherokee Road and Road 22 (Three Sisters Wildfire): There are two roads closed to the general public:... http://t.co/WwnL8FvfMM #IDFire
1
1
1
1
is_disaster: As California fires rage the Forest Service sounds the alarm about sharply rising wildfire costs http://t.co/ht8FyiMJlR
is_disas

is_disaster: Almost *wrecked* my van the other day because of this guy (yeah I brake &amp; also care for animals  ⁇  get used to it)! https://t.co/8ZaHavmtT6
is_disaster: 'What manner of human being would parcel out a baby as though it were a wrecked car in a junk yard? The ⁇ _'  ⁇  dtom2 http://t.co/lj6lzo4isX
1
0
0
0
is_disaster: @Nathan26_RFC thought you said Saturday night there and near died lolol you'll be wrecked!!
is_disaster: I just wanna ease your mind and make you feel alright.????
is_disaster: @yakubOObs think he deactivated because his notifications are aids after tesco wrecked him lol
is_disaster: RT CNBC '3 words from Disney CEO Bob Iger wrecked Disney's stock says Jim Cramer: http://t.co/f0texKsqhL http://t.co/ilySLaTMgI'
1
0
0
0
is_disaster: Smackdown tyme this should put me in a good mood again since it got wrecked smh
is_disaster: @thrillhho jsyk I haven't stopped thinking abt remus slumped against the bathroom door all day I was wrecked??????????
is_disaster: @stigh

In [17]:
submission = pd.concat([test_df[['id']], pd.Series(test_res)], axis=1)

submission = submission.rename({0:'target'}, axis=1)

In [18]:
submission.to_csv('kaggle_submissions/submission_1.csv', index=None)

In [19]:
check = pd.concat([pd.read_csv('kaggle_tweets_data/test.csv')[['id', 'text']], pd.Series(test_res)], axis=1)

In [20]:
check

,id,text,0
0,0,Just happened a terrible car crash,1
1,2,"Heard about #earthquake is different cities, stay safe everyone.",0
2,3,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",1
3,9,Apocalypse lighting. #Spokane #wildfires,1
4,11,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTENERS XrWn,0
3259,10865,Storm in RI worse than last hurricane. My city&amp;3others hardest hit. My yard looks like it was bombed. Around 20000K still without power,1
3260,10868,Green Line derailment in Chicago http://t.co/UtbXLcBIuY,1
3261,10874,MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3,1
